In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/crowdflower-search-relevance/train.csv.zip')
test = pd.read_csv('/kaggle/input/crowdflower-search-relevance/test.csv.zip')
train.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.describe()

In [ ]:
train['query'].map(lambda x:len(x.split())).value_counts()

In [ ]:
train['product_title'].map(lambda x:len(x.split())).value_counts()

In [ ]:
split = int(len(train)*0.8)
train_0, dev = train[:split], train[split:]

In [ ]:
clean_train_1 = train_0[train_0.relevance_variance <1].copy()
clean_train_2 = train_0[train_0.relevance_variance <0.50].copy()
dev.describe()

In [ ]:
clean_train_1.describe()

In [ ]:
clean_train_2.describe()

In [ ]:
## Skipping product description as it's too lengthy and missing values
train = clean_train_1
train_input = train.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
dev_input =  dev.apply(lambda x: x['query']+' '+x['product_title'], axis=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(ngram_range=(1, 5),stop_words = 'english', strip_accents='unicode')
train_x = tfidf.fit_transform(train_input)
dev_x = tfidf.transform(dev_input)

In [ ]:
train_y, dev_y = train.median_relevance.to_list(), dev.median_relevance.to_list()
train_y = [(x-1)/3 for x in train_y]
dev_y = [(x-1)/3 for x in dev_y]
np.mean(train_y), np.max(train_y), np.min(train_y)

In [ ]:
import datetime
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
time = f"{formatted_time}"

In [ ]:
import os
import wandb
from wandb.keras import WandbCallback
os.system('! wandb login be213aaff4ff14945d480abc18697d8664bba8c8')

In [ ]:
from sklearn.metrics import mean_squared_error, cohen_kappa_score, make_scorer
def reg_scorer(true, pred):
    pred = [min(1, max(0,x)) for x in pred]
    pred = [int(round((x*3)+1)) for x in pred]
    true = [int(round((x*3)+1)) for x in true]
    return cohen_kappa_score(true, pred)

In [ ]:
wandb.init(project='CrowdflowerModelsOnly', name='figuring out params '+time)

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
#clf = LinearRegression().fit(train_x, train_y)
#clf = SGDRegressor(verbose=1,n_iter_no_change=20).fit(train_x, train_y)
'''param_grid = {'C': [1], 'epsilon':[0.1,0.05], 'kernel': ('linear', 'rbf')}
svr  = SVR()
scorer = make_scorer(reg_scorer, greater_is_better=True)
clf = GridSearchCV(svr, param_grid, verbose=True,scoring=scorer, n_jobs=8)
clf.fit(train_x, train_y)
clf.best_estimator_, clf.best_params_, clf.best_score_'''

In [ ]:
## 0.26 is the best score till now

#preds = clf.best_estimator_.predict(dev_x)
#mean_squared_error(dev_y, preds),  reg_scorer(dev_y, preds)

In [ ]:
scorer = make_scorer(reg_scorer, greater_is_better=True)

#GridSearch
'''from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.utils import check_random_state

X_train, X_dev, y_train, y_dev = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# Define base models
svr = SVR()
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()


meta_model = LinearRegression()



# Perform GridSearchCV for each base model
'''
param_grid_svr = {'C': [0.1], 'epsilon': [0.01, 0.05], 'kernel': ['linear', 'rbf']}
param_grid_rf = {'n_estimators': [50, 200], 'max_depth': [None, 10]}
param_grid_gb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]}

'''

param_grid_svr = {'C': [0.1], 'epsilon': [0.01, 0.05], 'kernel': ['linear', 'rbf']}
param_grid_rf = {'n_estimators': [50], 'max_depth': [10]}
param_grid_gb = {'n_estimators': [100], 'learning_rate': [0.05]}

grid_search_svr = GridSearchCV(svr, param_grid_svr, verbose=True, scoring=scorer, n_jobs=-1)
grid_search_rf = GridSearchCV(rf, param_grid_rf, verbose=True, scoring=scorer, n_jobs=-1)
grid_search_gb = GridSearchCV(gb, param_grid_gb, verbose=True, scoring=scorer, n_jobs=-1)

grid_search_svr.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)
grid_search_gb.fit(X_train, y_train)

# Get best estimators
svr_best_estimator = grid_search_svr.best_estimator_
rf_best_estimator = grid_search_rf.best_estimator_
gb_best_estimator = grid_search_gb.best_estimator_
'''



In [ ]:
'''# Define stacking regressor with cross-validated predictions
stacked_reg = StackingRegressor(
    estimators=[('svr', svr), ('rf', rf), ('gb', gb)],
    final_estimator=meta_model,
    cv=5  
)'''
'''stacked_reg = StackingRegressor(
    estimators=[('svr', grid_search_svr.best_estimator_),
                ('rf', grid_search_rf.best_estimator_),
                ('gb', grid_search_gb.best_estimator_)],
    final_estimator=meta_model,
    cv=5  # 5-fold cross-validation for stacking
)
# Fit the stacking regressor with the best base models
stacked_reg.fit(X_train, y_train)

# Evaluate on the validation set
y_pred_stacked = stacked_reg.predict(X_dev)
mse_stacked = mean_squared_error(y_dev, y_pred_stacked)
print(f'Mean Squared Error for Stacked Ensemble: {mse_stacked}')'''

In [ ]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from scipy.stats import randint, uniform

# Assuming you have train_x, train_y as your training data
X_train, X_dev, y_train, y_dev = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# Define base models
svr = SVR()
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()
knn = KNeighborsRegressor()
lasso = Lasso()

# Define meta-model
meta_model = LinearRegression()

# Define parameter distributions for RandomizedSearchCV
#param_dist_svr = {'C': uniform(0.1, 1.0), 'epsilon': uniform(0.01, 0.1), 'kernel': ['linear', 'rbf']}
param_grid_svr = {'C': [0.1], 'epsilon': [0.01, 0.05], 'kernel': ['linear', 'rbf']}

#updated grid
'''param_grid = {
    'estimator__C': [0.001, 0.01, 0.1],
    'estimator__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'estimator__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
    'estimator__epsilon': [0.01, 0.05],
    'estimator__degree': [2, 3, 4, 5],
    'estimator__coef0': [0.0, 0.1, 0.5, 1.0],
    'estimator__shrinking': [True, False],
    'estimator__max_iter': [-1, 100, 500, 1000],
}'''

param_dist_rf = {'n_estimators': randint(50, 200), 'max_depth': [None, 10, 20]}
param_dist_gb = {'n_estimators': randint(50, 200), 'learning_rate': uniform(0.01, 0.2)}
param_dist_knn = {'n_neighbors': randint(1, 10), 'weights': ['uniform', 'distance']}
param_dist_lasso = {'alpha': uniform(0.1, 1.0)}

# Perform RandomizedSearchCV for each base model
#random_search_svr = RandomizedSearchCV(svr, param_distributions=param_dist_svr, n_iter=5, verbose=True, scoring=scorer, n_jobs=-1)
grid_search_svr = GridSearchCV(svr, param_grid_svr, verbose=True, scoring=scorer, cv=5, n_jobs=-1)
###

grid_search_svr.fit(X_train, y_train)

y_pred_svr = grid_search_svr.predict(X_dev)
mse_svr = mean_squared_error(y_dev, y_pred_svr)
#wandb.log(grid_search_svr.get_params())
#wandb.sklearn.log_model(grid_search_svr)
svr_params = grid_search_svr.get_params()
models = {
    "SVR": {
        "model": grid_search_svr,
        "params": svr_params,
        "score" : mse_svr,
        "best estimator": grid_search_svr.best_estimator_
    },
    "SVR": {
        "model": grid_search_svr,
        "params": svr_params,
        "score" : mse_svr
    }
}
wandb.config.update(models)

wandb.finish()

test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = grid_search_svr.best_estimator_.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)

####

'''random_search_rf = RandomizedSearchCV(rf, param_distributions=param_dist_rf, n_iter=5, verbose=True, scoring=scorer, n_jobs=-1)
random_search_gb = RandomizedSearchCV(gb, param_distributions=param_dist_gb, n_iter=5, verbose=True, scoring=scorer, n_jobs=-1)
random_search_knn = RandomizedSearchCV(knn, param_distributions=param_dist_knn, n_iter=5, verbose=True, scoring=scorer, n_jobs=-1)
random_search_lasso = RandomizedSearchCV(lasso, param_distributions=param_dist_lasso, n_iter=5, verbose=True, scoring=scorer, n_jobs=-1)

#random_search_svr.fit(X_train, y_train)
grid_search_svr.fit(X_train, y_train)
random_search_rf.fit(X_train, y_train)
random_search_gb.fit(X_train, y_train)
random_search_knn.fit(X_train, y_train)
random_search_lasso.fit(X_train, y_train)'''




In [ ]:
grid_search_svr.get_params()

In [ ]:
'''y_pred_svr = grid_search_svr.predict(X_dev)
mse_svr = mean_squared_error(y_dev, y_pred_svr)
#wandb.log(grid_search_svr.get_params())
#wandb.sklearn.log_model(grid_search_svr)
svr_params = grid_search_svr.get_params()
models = {
    "SVR": {
        "model": grid_search_svr,
        "params": svr_params,
        "score" : mse_svr
    },
    "SVR": {
        "model": grid_search_svr,
        "params": svr_params,
        "score" : mse_svr
    }
}
wandb.config.update(models)
#wandb.log(mse_svr)'''

In [ ]:
'''weights = {'svr': 2.0, 'rf': 0.5, 'gb': 1.0, 'knn': 1.0, 'lasso': 1.0}
#apparently we cant use weights here

stacked_reg = StackingRegressor(
    estimators=[
        ('svr', grid_search_svr.best_estimator_),
        #('rf', random_search_rf.best_estimator_),
        ('gb', random_search_gb.best_estimator_),
        ('knn', random_search_knn.best_estimator_),
        ('lasso', random_search_lasso.best_estimator_)
    ],
    final_estimator=meta_model,
    cv = 5,
    n_jobs=-1,
    passthrough=True,
    #weights=[weights[model_name] for model_name, _ in [('svr', svr), ('rf', rf), ('gb', gb), ('knn', knn), ('lasso', lasso)]]
)

# Fit the stacking regressor with the best base models
stacked_reg.fit(X_train, y_train)

# Evaluate on the validation set
y_pred_stacked = stacked_reg.predict(X_dev)
mse_stacked = mean_squared_error(y_dev, y_pred_stacked)
print(f'Mean Squared Error for Stacked Ensemble: {mse_stacked}')'''

In [ ]:
'''test_input =  test.apply(lambda x: x['query']+' '+x['product_title'], axis=1)
test_x = tfidf.transform(test_input)
pred = stacked_reg.predict(test_x)
pred = [min(1, max(0,x)) for x in pred]
pred = [int(round((x*3)+1)) for x in pred]
out = pd.DataFrame({"id": test.id.to_list(), "prediction": pred})
out.to_csv('submission.csv', index=False)'''

In [ ]:
sub = pd.read_csv('/kaggle/input/crowdflower-search-relevance/sampleSubmission.csv.zip')
sub


In [ ]:
out

In [ ]:
wandb.finish()